#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [119]:
import os, sys

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


In [120]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [121]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 1*365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [126]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())

# if repo_root == os.getcwd(): #then go one directory above to reach repo root
#     repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if "d:\\Work" in repo_root or "D:\\Work" in repo_root:
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


In [123]:

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["RELIANCE.NS", "HDFCBANK.NS", "TCS.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2025-10-18 01:22:44,325 - INFO - 📈 Fetching data for 2178 tickers...
2025-10-18 01:22:44,326 - INFO - Fetching data from 2024-09-18 to 2025-10-18


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[*********************100%***********************]  2178 of 2178 completed
2025-10-18 01:24:01,143 - ERROR - 
1 Failed download:
2025-10-18 01:24:01,143 - ERROR - ['NDTV-RE.NS']: YFInvalidPeriodError("NDTV-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
2025-10-18 01:24:04,386 - INFO - ✅ Download completed in 80.07 seconds
2025-10-18 01:24:08,751 - INFO - 📊 Final dataset: 561,107 rows from 2024-09-18 to 2025-10-17


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2024-09-18,20MICRONS.NS,NaN,290.189453,305.818880,288.895310,294.370583,209029.0,290.189453,290.189453,290.189453,290.189453,NaN,NaN,20MICRONS
1,2024-09-19,20MICRONS.NS,NaN,302.683014,305.619763,284.116868,294.669218,246937.0,291.379316,290.679397,290.436850,290.313767,NaN,NaN,20MICRONS
2,2024-09-20,20MICRONS.NS,NaN,300.293793,312.588274,298.651217,304.873117,289918.0,292.228314,291.056432,290.632037,290.413071,NaN,NaN,20MICRONS
3,2024-09-23,20MICRONS.NS,NaN,297.705475,306.067733,296.709971,303.180747,101611.0,292.749948,291.317179,290.772105,290.485632,NaN,NaN,20MICRONS
4,2024-09-24,20MICRONS.NS,NaN,299.348083,308.606259,298.651219,299.646723,188515.0,293.378342,291.632116,290.941927,290.573816,NaN,NaN,20MICRONS


##### Call Accumulation signal

In [124]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

✅ Database updated successfully


,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv
0,ORCHASP,2025-10-17,539287.3846,70.0,1.79,75.83,False,3.232374,3.004607,3.072430,3.660000,1.742380e+06,6.245476e+05
1,SKYGOLD,2025-10-17,360437.3428,70.0,1.47,75.58,False,309.128166,301.655654,314.913518,334.100006,1.601858e+06,9.739684e+05
2,ICICIGI,2025-10-17,256606.8770,65.0,1.33,64.58,False,1914.052257,1904.028565,1906.076819,2001.000000,1.466945e+06,8.163585e+05
3,ZENITHSTL,2025-10-17,102402.4510,45.0,1.78,73.11,False,8.108413,7.906219,8.082078,8.940000,4.305841e+05,1.960824e+05
4,SUBEXLTD,2025-10-17,95496.4982,35.0,1.63,69.03,False,13.042668,13.027522,15.385602,13.600000,3.381884e+06,1.479605e+06


#### EMA Signal indicator - TBC

##### Update the Stock returns table

In [125]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
# importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")


✅ Total signals loaded: 2049
✅ Total signals after merging: 273801
📦 Existing SIGNAL_RETURNS records: 1978
✅ New unique records found: 2
💾 SIGNAL_RETURNS table updated successfully.
🔒 Database connection closed.
✅ Signal returns generation successful!


,Symbol,Signal_Type,Signal_date,Signal_Price,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc
778,ICICIGI,Accumulation Signal,2025-10-17,2001.000000,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,0.0,2001.000000,2025-10-17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0
1472,PVSL,Accumulation Signal,2025-10-17,148.320007,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,0.0,148.320007,2025-10-17,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0
